<img src="figures/ampel.png" width="200">

### AMPEL and the Vera C. Rubin Observatory

The Vera Rubin Observatory, and the LSST survey, will provide a legacy collection of real-time data. Considering the potential long term impact of any transient programs, the AMPEL analysis platform was developed to 
host complex science program with provenance requirements matching those of the observatory. In essence, this means the creation of _scientific analysis schema_ which detail all scientific/algorithmic choices being made. This schema can be distributed with publications, and consistently applied to simulated, archived and real-time datasets.

##### This notebook : Real-time multi-messenger programs

This notebook presents options for real-time multi-messenger channels using the AMPEL framework. It is closely aligned with `Tutorial 3` from this repository.

### Native procesessing of heterogeneous multi-messenger data streams

Multi-messenger analysis by definition deals with processing of heterogenous data sources. We define _native_ multi-messenger data processing as one where multiple different streams are processed on equal footing, and downstream anaylsis have the flexibility to freely create events (states) from this data. _Non-native_ processing uses one stream as "master", to which external catalogs/watchlists/sky-maps are compared.

Native heterogenous stream processing in AMPEL is achieved through multiple controllers which independently filter and ingests different input streams. A T1 _Combine_ unit is used to create multi-messenger states which links datapoints from both streams.

The following sample analysis schema shows a setup where a public LSST stream and a private hop.SCIMMA streams are ingested.

```
...
controller:
  - unit: LSSTAlertStreamController         # Client for reader LSST alert stream
      processor:
        unit: AlertProcessor
          directives:
            filter:
              unit: OpticalDataFilter       # Filter LSST data based on minimal usage criteria
            t0_add:
              unit: ZiAlertContentIngester  # Ingest optical data
  - unit: HopskotchStreamController         # Wrapper around HOPSKOTCH client
      config:
        server: server_name                 
        topic: my_private_topic             # Topic for private neutrino alert distribution
        auth: SECRET
      processor:
        unit: AlertProcessor
          directives:
            filter:
              unit: NeutrinoDataFilter      # Minimal significance criteria for accepting neutrino alerts
            t0_add:
              unit: HsAlertContentIngester  # Ingest content of stream
t1_combine:
  unit: PhotoNeutrinoCompoundIngester       # Unit w. recipe for associating data
    config:
      time_tolerance: 42                    # Sample parameter: how much can the emission times vary
...
...
```

In this structure, scientific program choices are made in the filters as well as in the `PhotoNeutrinoCompoundIngester`.

### Non-native multi-messenger analysis

We here demo the use of an analysis T2 unit to compare selected optical transients with a catalog of recent multi-messenger events, ```T2MultiMessMatch```. 

In [ ]:
import os
%load_ext ampel_quick_import
%qi DevAmpelContext AmpelLogger T2Processor T3Processor ChannelModel AlertProcessor TarAlertLoader ChannelModel AbsAlertFilter T2MultiMessMatch

In [ ]:
T2MultiMessMatch??

A unit - `T2MultiMessMatch` - retrieves a recent list of potential multi-messenger events, together with information of their spatial, temporal and energetics properties. It will then return a combined "p-value" of how closely these match the properties of the optical transients. A set of scaling factors determine the relative weight between these dimensions. A real unit would read this information from a local mirror of e.g. LIGO/VIRGO or IceCube events, while we here use an invented event.

Path to unit: `Ampel-contrib-sample/ampel/contrib/sample/t2/T2MultiMessMatch.py`

The final step of this notebook is the selection of events we consider "good" matches, based on having a small p-value.

In [ ]:
AMPEL_CONF = "../../ampel_config.yml"
ALERT_ARCHIVE = '../sample_data/ztfpub_200917_pruned.tar.gz'

In [ ]:
# The operation context is created based on a setup configuration file.
# db_prefix sets the DB name to use
ctx = DevAmpelContext.load(
    config_file_path = AMPEL_CONF,
    db_prefix = "AmpelTutorial",
    purge_db = True,
)

In [ ]:
# The Unit parameters will define how 
mm_match_conf = {
    'temporal_pull_scaling' : 1,         # Neutral - we do not know when Neutrinos are emitted
    'spatial_pull_scaling'  : 3.,        # Reasonably sure regarding location
    'energy_pull_scaling'   : 0.001,     # Little constraint on energy, deweight this
    'match_where'           : 'latest',  # latest, first or mean
}
mm_match_config_id = ctx.add_config_id( mm_match_conf )

In [ ]:
# A scientific program, a channel, is added
ctx.add_channel(
    name="demo_SN09if",
    access=['ZTF', 'ZTF_PUB']
)

In [ ]:
# The channel is constructed from two units, each controlled by parameters. 
# Lets start with the straightforward filter 
filter_conf = {
    'min_rb':0.3,
    'min_ndet':7,
    'min_tspan':10,
    'max_tspan' : 200,
    'min_gal_lat':15,
}
filter_config_id = ctx.add_config_id( filter_conf )

In [ ]:
# The template matching has now been moved into a separate unit:
# T2SNcosmoComp
# where we added some configurability. 
match_conf = {
    'target_model_name':'v19-2009ip-corr', 
    'base_model_name':'salt2', 
    'chi2dof_cut':2.,
    'chicomp_scaling':0.5,
}
match_config_id = ctx.add_config_id( match_conf )

In [ ]:
# A channel can specify which streams to read, how these should be combined and what units
# should be run on each data combination.
# This is provided as directives to the AlertProcessor, which besides processing the alerts
# also submit tickets to the DB concerning further operations to execute for any transients
# that pass the initial filter stage.
ap = AlertProcessor(
    context = ctx,
    process_name = "ipyton_notebook_test",
    supplier = "ZiAlertSupplier",
    log_profile = "debug",
    directives = [
        {
            "channel": "demo_SN09if", 
            "filter": {"unit": "SimpleDecentFilterCopy","config": filter_config_id
                        },
            "stock_update": "ZiStockIngester",
            't0_add': {
                "ingester": "ZiAlertContentIngester",
                "t1_combine": [
                    {
                        "ingester": "PhotoCompoundIngester",
                        "config": {"combiner": "ZiT1Combiner"},
                        "t2_compute": {
                            "ingester": "PhotoT2Ingester",
                            "config": {"tags": ["ZTF"]},
                            "units": [
                                {'unit': 'T2SNcosmoComp',
                                 'config': match_config_id
                                },
                                {'unit': 'T2MultiMessMatch',
                                 'config': mm_match_config_id
                                },
                                
                            ]
                        }
                    }
                ],
            }
        }
    ]
)

In [ ]:
# Provide a link to the alert collection to use
ap.set_loader(TarAlertLoader(file_path=ALERT_ARCHIVE))

In [ ]:
ap.set_iter_max(1000)

In [ ]:
ap.run()

In [ ]:
t2p = T2Processor(context=ctx, process_name="T2Processor_test", log_profile="debug")

In [ ]:
t2p.run()

In deciding which targets to follow we wish to also make use of the match criteria. Can be done as follows:

In [ ]:
t3 = T3Processor(
    context=ctx,
    process_name = "T3Processor_test",
    log_profile = "default", # debug
    channel = "demo_SN09if",
    directives = [ {
        "select": {
            "unit": "T3FilteringStockSelector",
            "config": {
                't2_filter': { 
                    'all_of': [                
                        {
                            'unit': 'T2SNcosmoComp',
                            'match': {'target_match': True}
                        }, 
                        {
                            'unit': 'T2MultiMessMatch',
                            'match': {'best_match': {"$lt":1} }
                        },                         
                    ]
            } }
        },
        "load": {
            "unit": "T3SimpleDataLoader",
            "config": {
                "directives": ["TRANSIENT", "DATAPOINT", "COMPOUND", "T2RECORD"],
            }

        },
        "run": {
            "unit": "T3UnitRunner",
            "config": {
                "directives": [
                      {
                            "project": {
                                "unit": "T3ChannelProjector",
                                "config": {
                                    "channel": "demo_SN09if"
                                }
                            },
                            "execute": [
                                {
                                    "unit": "T3HelloWorld",
                                    "config": {
                                        't2info_from' : ['T2SNcosmoComp', 'T2MultiMessMatch']
                                    },
                                },
                            ]

                      }
                ]
            }
        }
    } ]
)

In [ ]:
t3.run()

Looks like ZTF20abyfpze is our target! Next step would be to propagate the coordinates to a suitable facility (or export for visual inspection).